In [1]:
from functools import wraps
import contextlib
import os
import time
import warnings
# warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
import scipy as sp
from sklearn import preprocessing

import tensorflow as tf
import tensorflow_probability as tfp

print(tf.__version__, tfp.__version__)

2.3.0 0.11.0


In [6]:
def add_grad_to_val(fn_obj):
  
  @wraps(fn_obj)
  def wrapped(x):
    return tfp.math.value_and_gradient(fn_obj, x)
  
  return wrapped

@contextlib.contextmanager
def time_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print("Evaluation took: {:.2f} seconds.".format(dt))

In [3]:
np.random.seed(10)

dim = 10
miniumum = np.ones([dim])
scales = np.exp(np.random.randn(dim))

@tf.function
@add_grad_to_val
def quadratic(x):
  return tf.reduce_sum(scales * (x - miniumum) ** 2, axis=-1)

start = np.random.randn(dim)

results = tfp.optimizer.lbfgs_minimize(quadratic, initial_position=tf.constant(start), tolerance=1e-6)

print('L-BFGS Results')
print('Converged:', results.converged)
print('Location of the minimum:', results.position)
print('Number of iterations:', results.num_iterations)

L-BFGS Results
Converged: tf.Tensor(True, shape=(), dtype=bool)
Location of the minimum: tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float64)
Number of iterations: tf.Tensor(10, shape=(), dtype=int32)


In [4]:
results = tfp.optimizer.bfgs_minimize(quadratic, initial_position=tf.constant(start), tolerance=1e-6)

print('BFGS Results')
print('Converged:', results.converged)
print('Location of the minimum:', results.position)
print('Number of iterations:', results.num_iterations)

BFGS Results
Converged: tf.Tensor(True, shape=(), dtype=bool)
Location of the minimum: tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float64)
Number of iterations: tf.Tensor(10, shape=(), dtype=int32)


In [5]:
data_path = tf.keras.utils.get_file(
  fname="prostate.data",
  origin="http://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data"
)

In [6]:
df_prostate = pd.read_csv(data_path, delim_whitespace=True, index_col=0)

In [7]:
np.random.seed(12345)

feature_names = ['lcavol', 'lweight',   'age',  'lbph', 'svi', 'lcp',   
                 'gleason', 'pgg45']

scalar = preprocessing.StandardScaler()
df_prostate[feature_names] = pd.DataFrame(
  scalar.fit_transform(
    df_prostate[feature_names].astype(np.float64)
  )
)

df_prostate_train = df_prostate.query("train=='T'")

features = df_prostate_train[feature_names]
labels = df_prostate_train[["lpsa"]]

feat = tf.constant(features.values, dtype=tf.float64)
lbl = tf.constant(labels.values, dtype=tf.float64)

dtype = feat.dtype

regularization = 0.0
dim = 8

start = np.random.randn(dim+1)


@tf.function(experimental_compile=True)
def regression_loss(params):
  params = tf.squeeze(params)
  intercept, beta = params[0], params[1:]
  
  pred = tf.matmul(feat, tf.expand_dims(beta, axis=-1)) + intercept
  
  mse_loss = tf.reduce_sum(
    tf.cast(
      tf.losses.mean_squared_error(y_true=lbl, y_pred=pred),
      tf.float64
    )
  )
  
  l1_penalty = regularization * tf.reduce_sum(tf.abs(beta))
  
  total_loss = mse_loss + l1_penalty
  return total_loss

In [8]:
results = tfp.optimizer.lbfgs_minimize(
  add_grad_to_val(regression_loss),
  initial_position=tf.constant(start),
  tolerance=1e-8
)

minimum = results.position
fitted_intercept = minimum[0]
fitted_beta = minimum[1:]

print('L-BFGS Results')
print('Converged:', results.converged)
print('Intercept: Fitted ({})'.format(fitted_intercept))
print('Beta:      Fitted {}'.format(fitted_beta))

L-BFGS Results
Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (2.3879985744556484)
Beta:      Fitted [ 0.68626215  0.28193532 -0.17030254  0.10799274  0.33634988 -0.24888523
  0.11992237  0.08689026]


In [9]:
initial_vertex = tf.expand_dims(tf.constant(start, dtype=dtype), axis=-1)

results = tfp.optimizer.nelder_mead_minimize(
  regression_loss,
  initial_vertex=initial_vertex,
  func_tolerance=1e-10,
  position_tolerance=1e-10
)

minimum = tf.squeeze(results.position)
fitted_intercept = minimum[0]
fitted_beta = minimum[1:]

print('Nelder Mead Results')
print('Converged:', results.converged)
print('Intercept: Fitted ({})'.format(fitted_intercept))
print('Beta:      Fitted {}'.format(fitted_beta))

Nelder Mead Results
Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (2.387998456121595)
Beta:      Fitted [ 0.68626266  0.28193456 -0.17030291  0.10799375  0.33635132 -0.24888703
  0.11992244  0.08689023]


In [10]:
np.random.seed(12345)

dim = 5
n_obs = 10000

beta = np.random.randn(dim)
intercept = np.random.randn()

features = np.random.randn(n_obs, dim)
probs = sp.special.expit(
  np.matmul(features, np.expand_dims(beta, -1)) + intercept
)

labels = sp.stats.bernoulli.rvs(probs)

regularization = 0.8
feat = tf.constant(features)
lbl = tf.constant(labels, dtype=feat.dtype)


@tf.function(experimental_compile=True)
@add_grad_to_val
def negative_log_likelihood(params):
  
  intercept, beta = params[0], params[1:]
  logit = tf.matmul(feat, tf.expand_dims(beta, -1)) + intercept
  
  log_likelihood = tf.reduce_sum(
    tf.nn.sigmoid_cross_entropy_with_logits(labels=lbl, logits=logit)
  )
  
  l2_penalty = regularization * tf.reduce_sum(tf.square(beta))
  
  total_loss = log_likelihood + l2_penalty
  return total_loss

start = np.random.randn(dim+1)

results = tfp.optimizer.lbfgs_minimize(
  negative_log_likelihood,
  initial_position=tf.constant(start),
  tolerance=1e-8
)

minimum = results.position
fitted_intercept = minimum[0]
fitted_beta = minimum[1:]

print('Converged:', results.converged)
print('Intercept: Fitted ({}), Actual ({})'.format(fitted_intercept, intercept))
print('Beta:\n\tFitted {},\n\tActual {}'.format(fitted_beta, beta))

Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (1.4111415084244363), Actual (1.3934058329729904)
Beta:
	Fitted [-0.18016612  0.53121578 -0.56420632 -0.5336374   2.00499675],
	Actual [-0.20470766  0.47894334 -0.51943872 -0.5557303   1.96578057]


In [3]:
class Logit:
  def __init__(self, dim=5, n_obs=10000):
    self.dim = dim
    self.need_compile = None
    self.reset(dim, n_obs)
    
  def reset(self, dim, n_obs):
    self.beta = np.random.randn(dim)
    self.intercept = np.random.randn()

    features = np.random.randn(n_obs, dim)
    probs = sp.special.expit(
      np.matmul(features, np.expand_dims(self.beta, -1)) + self.intercept
    )
    
    labels = sp.stats.bernoulli.rvs(probs)

    self.regularization = tf.constant(0.8, dtype=tf.float64)
    self.feat = tf.constant(features)
    self.lbl = tf.constant(labels, dtype=tf.float64)
    
    self.dim = dim
    self.need_compile = True
    
  def negative_log_likelihood(self, params):
    intercept, beta = params[0], params[1:]
    logit = tf.matmul(self.feat, tf.expand_dims(beta, -1)) + intercept
  
    log_likelihood = tf.reduce_sum(
      tf.nn.sigmoid_cross_entropy_with_logits(labels=self.lbl, logits=logit)
    )

    l2_penalty = self.regularization * tf.reduce_sum(tf.square(beta))
  
    total_loss = log_likelihood + l2_penalty
    return total_loss, tf.gradients(total_loss, params)[0]
  
  def optimize(self):
    if self.need_compile:
      self.fn_obj = tf.function(self.negative_log_likelihood, experimental_compile=True)
      self.need_compile = False
      
    start = np.random.randn(self.dim+1)
      
    with time_execution():
      res = tfp.optimizer.lbfgs_minimize(
        self.fn_obj,
        initial_position=tf.constant(start),
        tolerance=1e-8
      )
    
    minimum = res.position
    fitted_intercept = minimum[0]
    fitted_beta = minimum[1:]

    print('Converged:', res.converged)
    print('Intercept: Fitted ({}), Actual ({})'.format(fitted_intercept, self.intercept))
    print('Beta:\n\tFitted {},\n\tActual {}'.format(fitted_beta, self.beta))
    
    return 

In [4]:
logit = Logit(5, 10000)

In [7]:
logit.optimize()

Evaluation took: 0.40 seconds.
Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (-0.3072084564496592), Actual (-0.31728045144336947)
Beta:
	Fitted [-0.70948868 -0.80968872  0.20237463  1.75168253  1.21557915],
	Actual [-0.71896628 -0.78084514  0.2158353   1.82554808  1.22479961]


In [8]:
logit.reset(6, 12000)

In [9]:
n = 20
while n>0:
  logit.optimize()
  n -= 1

Evaluation took: 0.31 seconds.
Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (1.6116725639111233), Actual (1.6092357945939424)
Beta:
	Fitted [ 0.21149472 -0.2529195  -1.05066718 -0.57306566  1.29443535 -0.02606926],
	Actual [ 0.20281719 -0.23340786 -1.0128432  -0.57315357  1.31181866  0.03572824]
Evaluation took: 0.13 seconds.
Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (1.611672563908161), Actual (1.6092357945939424)
Beta:
	Fitted [ 0.21149472 -0.2529195  -1.05066718 -0.57306566  1.29443535 -0.02606926],
	Actual [ 0.20281719 -0.23340786 -1.0128432  -0.57315357  1.31181866  0.03572824]
Evaluation took: 0.15 seconds.
Converged: tf.Tensor(True, shape=(), dtype=bool)
Intercept: Fitted (1.6116725639084013), Actual (1.6092357945939424)
Beta:
	Fitted [ 0.21149472 -0.2529195  -1.05066718 -0.57306566  1.29443535 -0.02606926],
	Actual [ 0.20281719 -0.23340786 -1.0128432  -0.57315357  1.31181866  0.03572824]
Evaluation took: 0.13 seconds.
Converged: tf